In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline
pd.options.display.float_format = '{:,.2f}'.format
plt.style.use('seaborn-white')
# colorblind safe
plt.style.use('seaborn-colorblind')
plt.style.use('tableau-colorblind10')

# width and precision for f strings
width = 10
precision = 4

# default sizes for plots
# https://matplotlib.org/3.3.0/tutorials/introductory/customizing.html#customizing-with-matplotlibrc-files
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 16
plt.rcParams['legend.fontsize'] = 'large'
plt.rcParams['figure.titlesize'] = 'medium'
plt.rcParams['lines.linewidth'] = 2

# other settings
pd.options.display.float_format = '{:,.4f}'.format
pd.set_option("display.precision", 3)
np.set_printoptions(precision=3, suppress=True)
%load_ext autoreload
%autoreload 2
pd.set_option('display.max_columns', None)
%config IPCompleter.greedy=True


In [2]:
# get data
USdata = pd.read_csv('data/iOverlander Places -  2020-08-22.csv')

In [19]:
USdata.loc[1699:1701,:]

,Location,Name,Category,Description,Latitude,Longitude,Altitude,Date verified,Open,Electricity,Wifi,Kitchen,Parking,Restaurant,Showers,Water,Toilets,Big rig friendly,Tent friendly,Pet friendly,Sanitation dump station,Outdoor gear,Groceries,Artesian goods,Bakery,Rarity in this area,Repairs vehicles,Repairs motorcycles,Repairs bicycles,Sells parts,Recycles batteries,Recycles oil,Bio fuel,Electric vehicle charging,Composting sawdust,Recycling center
1699,NaN,Coachella Vista Point,Wild Camping,A little chunk of BLM land sitting just above ...,33.6482,-116.4004,727.0000,2020-04-14 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,No,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
1700,"Cam Playa, Aguada, 00602, Puerto Rico",Pico de Piedra Beach,Informal Campsite,"Nice and Beautiful west coast beach , Great Re...",18.3842,-67.2122,nan,2017-12-22 00:00:00 UTC,Yes,Yes,Yes - Average,Yes,nan,Yes,Warm,Potable,Running Water,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
1701,NaN,Under the Bridge,Informal Campsite,Several parking spaces (for cars) under/next t...,30.2885,-81.4195,0.1891,2020-05-16 00:00:00 UTC,Yes,No,No,No,nan,No,Warm,No,Running Water,No,No,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
USdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8790 entries, 0 to 8789
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Location                   1413 non-null   object 
 1   Name                       8790 non-null   object 
 2   Category                   8790 non-null   object 
 3   Description                8790 non-null   object 
 4   Latitude                   8790 non-null   float64
 5   Longitude                  8790 non-null   float64
 6   Altitude                   8567 non-null   float64
 7   Date verified              8790 non-null   object 
 8   Open                       8790 non-null   object 
 9   Electricity                7987 non-null   object 
 10  Wifi                       7987 non-null   object 
 11  Kitchen                    7987 non-null   object 
 12  Parking                    0 non-null      float64
 13  Restaurant                 7987 non-null   objec

In [3]:
import os
G_MAPS_API = os.environ.get("GOOGLE_STATICMAPS_API")

In [7]:
import random, time

In [4]:
import googlemaps

In [5]:
gmaps = googlemaps.Client(G_MAPS_API)

In [6]:
n_requests = 0

In [31]:
def get_state_zip(df):
    '''
    gets zip code and state and inserts into df
    '''
    zips = []
    states = []
    
    global n_requests
    
    for i, site in df.iterrows():
        result = None
        print(f'Pulling request {i}, total API requests so far = {n_requests}')
        
        # get latlong in right format
        # temp save coords
        lat = str(site['Latitude'])
        long = str(site['Longitude'])
        latlong = lat + ',' + long
        #rint(latlong)
        # get geocode data
        res_type = 'postal_code' # administrative_area_level_1|
        result = gmaps.reverse_geocode(latlong, result_type=res_type)
        # increase counter
        n_requests += 1
        #rint(result)
        if result:
            # pull out things we need
            zip_code = result[0]['address_components'][0]['short_name']
            # sometimes the index isn't the same
            #state = result[0]['address_components'][3]['short_name']
            types = ['administrative_area_level_1', 'political']
            statethingy = [d.items() for d in result[0]['address_components'] if d['types'] == types]
            # sometimes there isn't this type (like Puerto Rico)
            #print(statethingy)
            if statethingy:
                state = [x[1] for x in statethingy[0] if x[0] == 'short_name'][0]
    #             for item in statethingy[0]:
    #                 if item[0] == 'short_name':
    #                     state = item[1]
            else:
                # if it didn't work for this row
                zip_code = ''
                state = ''
        else:
            # if it didn't work for this row
            zip_code = ''
            state = ''
        
        # append
        zips.append(zip_code)
        states.append(state)
        
        # wait a bit before next request
        wait_time = random.randint(1, 3)
        print(f'waiting for: {wait_time} seconds')
        time.sleep(wait_time) # in seconds
    
    return zips, states

In [14]:
# if we have a location we don't need to get it
USdata_need_loc = USdata[pd.isna(USdata['Location'])].copy()

In [15]:
len(USdata_need_loc)

7377

In [ ]:
# if we have location, we can assign states like this:
# USdata_CO = USdata_CO[USdata_CO['Location'].str.contains(' CO', case=True, regex=False)]

In [34]:
# do first 500 for now
USdata_500 = USdata_need_loc.copy().head(500)

In [35]:
USdata_500.head()

,index,Location,Name,Category,Description,Latitude,Longitude,Altitude,Date verified,Open,Electricity,Wifi,Kitchen,Parking,Restaurant,Showers,Water,Toilets,Big rig friendly,Tent friendly,Pet friendly,Sanitation dump station,Outdoor gear,Groceries,Artesian goods,Bakery,Rarity in this area,Repairs vehicles,Repairs motorcycles,Repairs bicycles,Sells parts,Recycles batteries,Recycles oil,Bio fuel,Electric vehicle charging,Composting sawdust,Recycling center
0,1221,NaN,Springer Lake,Wild Camping,Dispersed camping on shore of small lake. Ceme...,36.4088,-104.6526,"1,788.0000",2020-04-01 00:00:00 UTC,Yes,No,No,No,nan,No,No,Natural Source,Pit Toilets,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
1,1222,NaN,"large flat spot - on route 9 , before the 89 j...",Wild Camping,Be careful low branches. Not suitable for rvs...,37.2615,-112.7791,"1,811.0000",2019-05-29 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
2,1223,NaN,Zigzag falls,Wild Camping,Lots of small camping spots along Kiwanis Camp...,45.3141,-121.8082,944.3531,2019-06-02 00:00:00 UTC,Yes,No,No,No,nan,No,No,Natural Source,No,No,Yes,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
3,1224,NaN,Topsy Grade Rd,Wild Camping,Open woodlands along the reservoir with plenty...,42.1318,-122.0332,"1,194.4113",2019-09-27 00:00:00 UTC,Yes,No,No,No,nan,No,No,Non-Potable,No,No,Yes,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
4,1225,NaN,Lava tubes,Wild Camping,Pit toilet picnic table site,34.9446,-108.1069,"2,324.8057",2017-09-24 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,Pit Toilets,Unknown,Unknown,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# test with problem one
problems = USdata.loc[1698:1702,:]

In [32]:
zips, states = get_state_zip(problems)

Pulling request 1698, total API requests so far = 1722
waiting for: 2 seconds
Pulling request 1699, total API requests so far = 1723
waiting for: 2 seconds
Pulling request 1700, total API requests so far = 1724
waiting for: 2 seconds
Pulling request 1701, total API requests so far = 1725
waiting for: 1 seconds


KeyboardInterrupt: 

In [28]:
zips

['98901', '92561', '', '32250', '90755']

In [36]:
zips, states = get_state_zip(USdata_500)

Pulling request 0, total API requests so far = 1739
waiting for: 2 seconds
Pulling request 1, total API requests so far = 1740
waiting for: 1 seconds
Pulling request 2, total API requests so far = 1741
waiting for: 3 seconds
Pulling request 3, total API requests so far = 1742
waiting for: 2 seconds
Pulling request 4, total API requests so far = 1743
waiting for: 2 seconds
Pulling request 5, total API requests so far = 1744
waiting for: 3 seconds
Pulling request 6, total API requests so far = 1745
waiting for: 2 seconds
Pulling request 7, total API requests so far = 1746
waiting for: 2 seconds
Pulling request 8, total API requests so far = 1747
waiting for: 2 seconds
Pulling request 9, total API requests so far = 1748
waiting for: 1 seconds
Pulling request 10, total API requests so far = 1749
waiting for: 2 seconds
Pulling request 11, total API requests so far = 1750
waiting for: 3 seconds
Pulling request 12, total API requests so far = 1751
waiting for: 3 seconds
Pulling request 13, to

Pulling request 108, total API requests so far = 1847
waiting for: 3 seconds
Pulling request 109, total API requests so far = 1848
waiting for: 1 seconds
Pulling request 110, total API requests so far = 1849
waiting for: 2 seconds
Pulling request 111, total API requests so far = 1850
waiting for: 2 seconds
Pulling request 112, total API requests so far = 1851
waiting for: 1 seconds
Pulling request 113, total API requests so far = 1852
waiting for: 3 seconds
Pulling request 114, total API requests so far = 1853
waiting for: 2 seconds
Pulling request 115, total API requests so far = 1854
waiting for: 2 seconds
Pulling request 116, total API requests so far = 1855
waiting for: 2 seconds
Pulling request 117, total API requests so far = 1856
waiting for: 3 seconds
Pulling request 118, total API requests so far = 1857
waiting for: 2 seconds
Pulling request 119, total API requests so far = 1858
waiting for: 3 seconds
Pulling request 120, total API requests so far = 1859
waiting for: 2 seconds

waiting for: 1 seconds
Pulling request 215, total API requests so far = 1954
waiting for: 3 seconds
Pulling request 216, total API requests so far = 1955
waiting for: 2 seconds
Pulling request 217, total API requests so far = 1956
waiting for: 1 seconds
Pulling request 218, total API requests so far = 1957
waiting for: 1 seconds
Pulling request 219, total API requests so far = 1958
waiting for: 2 seconds
Pulling request 220, total API requests so far = 1959
waiting for: 1 seconds
Pulling request 221, total API requests so far = 1960
waiting for: 3 seconds
Pulling request 222, total API requests so far = 1961
waiting for: 3 seconds
Pulling request 223, total API requests so far = 1962
waiting for: 2 seconds
Pulling request 224, total API requests so far = 1963
waiting for: 1 seconds
Pulling request 225, total API requests so far = 1964
waiting for: 3 seconds
Pulling request 226, total API requests so far = 1965
waiting for: 2 seconds
Pulling request 227, total API requests so far = 1966

Pulling request 322, total API requests so far = 2061
waiting for: 2 seconds
Pulling request 323, total API requests so far = 2062
waiting for: 1 seconds
Pulling request 324, total API requests so far = 2063
waiting for: 3 seconds
Pulling request 325, total API requests so far = 2064
waiting for: 2 seconds
Pulling request 326, total API requests so far = 2065
waiting for: 3 seconds
Pulling request 327, total API requests so far = 2066
waiting for: 3 seconds
Pulling request 328, total API requests so far = 2067
waiting for: 2 seconds
Pulling request 329, total API requests so far = 2068
waiting for: 3 seconds
Pulling request 330, total API requests so far = 2069
waiting for: 2 seconds
Pulling request 331, total API requests so far = 2070
waiting for: 1 seconds
Pulling request 332, total API requests so far = 2071
waiting for: 3 seconds
Pulling request 333, total API requests so far = 2072
waiting for: 3 seconds
Pulling request 334, total API requests so far = 2073
waiting for: 3 seconds

Pulling request 429, total API requests so far = 2168
waiting for: 1 seconds
Pulling request 430, total API requests so far = 2169
waiting for: 1 seconds
Pulling request 431, total API requests so far = 2170
waiting for: 1 seconds
Pulling request 432, total API requests so far = 2171
waiting for: 2 seconds
Pulling request 433, total API requests so far = 2172
waiting for: 2 seconds
Pulling request 434, total API requests so far = 2173
waiting for: 1 seconds
Pulling request 435, total API requests so far = 2174
waiting for: 1 seconds
Pulling request 436, total API requests so far = 2175
waiting for: 3 seconds
Pulling request 437, total API requests so far = 2176
waiting for: 1 seconds
Pulling request 438, total API requests so far = 2177
waiting for: 3 seconds
Pulling request 439, total API requests so far = 2178
waiting for: 2 seconds
Pulling request 440, total API requests so far = 2179
waiting for: 1 seconds
Pulling request 441, total API requests so far = 2180
waiting for: 2 seconds

In [37]:
len(zips)

500

In [38]:
len(states)

500

In [39]:
# save
zips_pd = pd.Series(zips, name='zip_codes') 
states_pd = pd.Series(states, name='states') 
zips_pd.to_csv('data/zip_codes.csv')
states_pd.to_csv('data/states.csv')

In [40]:
USdata_zip_states = USdata_500.copy()

In [41]:
USdata_zip_states['zip_code'] = zips
USdata_zip_states['State'] = states

In [42]:
USdata_zip_states.head()

,index,Location,Name,Category,Description,Latitude,Longitude,Altitude,Date verified,Open,Electricity,Wifi,Kitchen,Parking,Restaurant,Showers,Water,Toilets,Big rig friendly,Tent friendly,Pet friendly,Sanitation dump station,Outdoor gear,Groceries,Artesian goods,Bakery,Rarity in this area,Repairs vehicles,Repairs motorcycles,Repairs bicycles,Sells parts,Recycles batteries,Recycles oil,Bio fuel,Electric vehicle charging,Composting sawdust,Recycling center,zip_code,State
0,1221,NaN,Springer Lake,Wild Camping,Dispersed camping on shore of small lake. Ceme...,36.4088,-104.6526,"1,788.0000",2020-04-01 00:00:00 UTC,Yes,No,No,No,nan,No,No,Natural Source,Pit Toilets,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,87747,NM
1,1222,NaN,"large flat spot - on route 9 , before the 89 j...",Wild Camping,Be careful low branches. Not suitable for rvs...,37.2615,-112.7791,"1,811.0000",2019-05-29 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,84755,UT
2,1223,NaN,Zigzag falls,Wild Camping,Lots of small camping spots along Kiwanis Camp...,45.3141,-121.8082,944.3531,2019-06-02 00:00:00 UTC,Yes,No,No,No,nan,No,No,Natural Source,No,No,Yes,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,97041,OR
3,1224,NaN,Topsy Grade Rd,Wild Camping,Open woodlands along the reservoir with plenty...,42.1318,-122.0332,"1,194.4113",2019-09-27 00:00:00 UTC,Yes,No,No,No,nan,No,No,Non-Potable,No,No,Yes,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,97627,OR
4,1225,NaN,Lava tubes,Wild Camping,Pit toilet picnic table site,34.9446,-108.1069,"2,324.8057",2017-09-24 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,Pit Toilets,Unknown,Unknown,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,87020,NM


In [43]:
USdata_CO = USdata_zip_states[USdata_zip_states['State'] == 'CO'].copy()

In [44]:
USdata_CO.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 6 to 488
Data columns (total 39 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   index                      16 non-null     int64  
 1   Location                   0 non-null      object 
 2   Name                       16 non-null     object 
 3   Category                   16 non-null     object 
 4   Description                16 non-null     object 
 5   Latitude                   16 non-null     float64
 6   Longitude                  16 non-null     float64
 7   Altitude                   16 non-null     float64
 8   Date verified              16 non-null     object 
 9   Open                       16 non-null     object 
 10  Electricity                15 non-null     object 
 11  Wifi                       15 non-null     object 
 12  Kitchen                    15 non-null     object 
 13  Parking                    0 non-null      float64


In [45]:
USdata_500.tail()

,index,Location,Name,Category,Description,Latitude,Longitude,Altitude,Date verified,Open,Electricity,Wifi,Kitchen,Parking,Restaurant,Showers,Water,Toilets,Big rig friendly,Tent friendly,Pet friendly,Sanitation dump station,Outdoor gear,Groceries,Artesian goods,Bakery,Rarity in this area,Repairs vehicles,Repairs motorcycles,Repairs bicycles,Sells parts,Recycles batteries,Recycles oil,Bio fuel,Electric vehicle charging,Composting sawdust,Recycling center
495,1862,NaN,"Veterans Memorial Park and area, Lake Charles",Informal Campsite,A beautiful park on the Lake Charles waterfron...,30.2313,-93.2212,44.0000,2020-02-25 00:00:00 UTC,Yes,Yes - Not at Sites,Yes - Slow,No,nan,No,No,Yes,Running Water,No,No,No,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
496,1863,NaN,Street near beach,Wild Camping,Just a street where u can park for free as lon...,34.0290,-118.5142,36.8156,2018-10-03 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,Yes,No,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
497,1866,NaN,Off of hole in the rock road,Wild Camping,"Next to a gated cattle run. Somewhat flat, but...",37.6367,-111.4475,"1,613.5075",2020-05-03 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,Yes,No,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
498,1867,NaN,Free campsite in Coconico NF,Wild Camping,Free campsite. No amenities. Very quiet.,35.4160,-111.4000,"1,707.8994",2018-04-11 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,Yes,Yes,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
499,1868,NaN,Chevron gas station with very large paved over...,Informal Campsite,Football size paved parking used by large RV a...,35.4607,-114.9171,"1,084.3755",2018-04-13 00:00:00 UTC,Yes,No,No,No,nan,Yes,Unknown,No,Running Water,Yes,No,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
USdata_501_1000 = USdata_need_loc.iloc[500:1000,:]

In [51]:
len(USdata_501_1000)

500

In [52]:
zips, states = get_state_zip(USdata_501_1000)

Pulling request 1869, total API requests so far = 2239
waiting for: 3 seconds
Pulling request 1870, total API requests so far = 2240
waiting for: 1 seconds
Pulling request 1871, total API requests so far = 2241
waiting for: 3 seconds
Pulling request 1872, total API requests so far = 2242
waiting for: 1 seconds
Pulling request 1873, total API requests so far = 2243
waiting for: 3 seconds
Pulling request 1875, total API requests so far = 2244
waiting for: 3 seconds
Pulling request 1876, total API requests so far = 2245
waiting for: 2 seconds
Pulling request 1877, total API requests so far = 2246
waiting for: 3 seconds
Pulling request 1878, total API requests so far = 2247
waiting for: 2 seconds
Pulling request 1879, total API requests so far = 2248
waiting for: 2 seconds
Pulling request 1880, total API requests so far = 2249
waiting for: 3 seconds
Pulling request 1881, total API requests so far = 2250
waiting for: 2 seconds
Pulling request 1882, total API requests so far = 2251
waiting f

Pulling request 2007, total API requests so far = 2345
waiting for: 1 seconds
Pulling request 2008, total API requests so far = 2346
waiting for: 2 seconds
Pulling request 2009, total API requests so far = 2347
waiting for: 3 seconds
Pulling request 2010, total API requests so far = 2348
waiting for: 3 seconds
Pulling request 2011, total API requests so far = 2349
waiting for: 2 seconds
Pulling request 2012, total API requests so far = 2350
waiting for: 3 seconds
Pulling request 2013, total API requests so far = 2351
waiting for: 3 seconds
Pulling request 2014, total API requests so far = 2352
waiting for: 1 seconds
Pulling request 2015, total API requests so far = 2353
waiting for: 3 seconds
Pulling request 2016, total API requests so far = 2354
waiting for: 1 seconds
Pulling request 2017, total API requests so far = 2355
waiting for: 3 seconds
Pulling request 2018, total API requests so far = 2356
waiting for: 3 seconds
Pulling request 2019, total API requests so far = 2357
waiting f

Pulling request 2121, total API requests so far = 2451
waiting for: 2 seconds
Pulling request 2122, total API requests so far = 2452
waiting for: 3 seconds
Pulling request 2123, total API requests so far = 2453
waiting for: 1 seconds
Pulling request 2124, total API requests so far = 2454
waiting for: 2 seconds
Pulling request 2125, total API requests so far = 2455
waiting for: 1 seconds
Pulling request 2126, total API requests so far = 2456
waiting for: 1 seconds
Pulling request 2127, total API requests so far = 2457
waiting for: 3 seconds
Pulling request 2128, total API requests so far = 2458
waiting for: 3 seconds
Pulling request 2129, total API requests so far = 2459
waiting for: 3 seconds
Pulling request 2130, total API requests so far = 2460
waiting for: 2 seconds
Pulling request 2131, total API requests so far = 2461
waiting for: 3 seconds
Pulling request 2132, total API requests so far = 2462
waiting for: 3 seconds
Pulling request 2133, total API requests so far = 2463
waiting f

Pulling request 2229, total API requests so far = 2557
waiting for: 1 seconds
Pulling request 2230, total API requests so far = 2558
waiting for: 3 seconds
Pulling request 2231, total API requests so far = 2559
waiting for: 2 seconds
Pulling request 2232, total API requests so far = 2560
waiting for: 1 seconds
Pulling request 2233, total API requests so far = 2561
waiting for: 1 seconds
Pulling request 2234, total API requests so far = 2562
waiting for: 3 seconds
Pulling request 2235, total API requests so far = 2563
waiting for: 2 seconds
Pulling request 2236, total API requests so far = 2564
waiting for: 2 seconds
Pulling request 2237, total API requests so far = 2565
waiting for: 3 seconds
Pulling request 2238, total API requests so far = 2566
waiting for: 3 seconds
Pulling request 2239, total API requests so far = 2567
waiting for: 3 seconds
Pulling request 2240, total API requests so far = 2568
waiting for: 2 seconds
Pulling request 2241, total API requests so far = 2569
waiting f

Pulling request 2335, total API requests so far = 2663
waiting for: 3 seconds
Pulling request 2336, total API requests so far = 2664
waiting for: 3 seconds
Pulling request 2337, total API requests so far = 2665
waiting for: 2 seconds
Pulling request 2338, total API requests so far = 2666
waiting for: 2 seconds
Pulling request 2339, total API requests so far = 2667
waiting for: 2 seconds
Pulling request 2340, total API requests so far = 2668
waiting for: 1 seconds
Pulling request 2341, total API requests so far = 2669
waiting for: 1 seconds
Pulling request 2342, total API requests so far = 2670
waiting for: 2 seconds
Pulling request 2343, total API requests so far = 2671
waiting for: 3 seconds
Pulling request 2344, total API requests so far = 2672
waiting for: 2 seconds
Pulling request 2345, total API requests so far = 2673
waiting for: 3 seconds
Pulling request 2346, total API requests so far = 2674
waiting for: 2 seconds
Pulling request 2347, total API requests so far = 2675
waiting f

In [53]:
zips

['31401',
 '70460',
 '26296',
 '26296',
 '26264',
 '',
 '89406',
 '95043',
 '87322',
 '89301',
 '89510',
 '87499',
 '84535',
 '97907',
 '04042',
 '08865',
 '92358',
 '85741',
 '82930',
 '86335',
 '86327',
 '90265',
 '93240',
 '37201',
 '28721',
 '22015',
 '27410',
 '85326',
 '96056',
 '03584',
 '87401',
 '22664',
 '87821',
 '86336',
 '80504',
 '70124',
 '22192',
 '78418',
 '13209',
 '85192',
 '89040',
 '',
 '84745',
 '99350',
 '93265',
 '86023',
 '',
 '93243',
 '93274',
 '95380',
 '14569',
 '15219',
 '83213',
 '',
 '74834',
 '81146',
 '81122',
 '93449',
 '94706',
 '77984',
 '97049',
 '98331',
 '99348',
 '88061',
 '08550',
 '84741',
 '17013',
 '89039',
 '97041',
 '60616',
 '58563',
 '87823',
 '20560',
 '85326',
 '17101',
 '63366',
 '92364',
 '75670',
 '82838',
 '81321',
 '28531',
 '90069',
 '82711',
 '42029',
 '62656',
 '32566',
 '84531',
 '07069',
 '81505',
 '30545',
 '36804',
 '08332',
 '89406',
 '98503',
 '29069',
 '',
 '08008',
 '89310',
 '74562',
 '49014',
 'N2C 1W9',
 '28771',
 '9

In [54]:
# save
zips_pd_next500 = pd.Series(zips, name='zip_codes') 
states_pd_next500 = pd.Series(states, name='states') 
zips_pd_next500.to_csv('data/zip_codes_next500.csv')
states_pd_next500.to_csv('data/states_next500.csv')

In [57]:
zips_pd_next500

0      31401
1      70460
2      26296
3      26296
4      26264
       ...  
495    59486
496         
497    89408
498    99603
499    97465
Name: zip_codes, Length: 500, dtype: object

In [58]:
USdata_501_1000['zip_code'] = zips
USdata_501_1000['State'] = states

/usr/local/anaconda3/envs/py377_maps/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/anaconda3/envs/py377_maps/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [59]:
USdata_501_1000.head()

,Location,Name,Category,Description,Latitude,Longitude,Altitude,Date verified,Open,Electricity,Wifi,Kitchen,Parking,Restaurant,Showers,Water,Toilets,Big rig friendly,Tent friendly,Pet friendly,Sanitation dump station,Outdoor gear,Groceries,Artesian goods,Bakery,Rarity in this area,Repairs vehicles,Repairs motorcycles,Repairs bicycles,Sells parts,Recycles batteries,Recycles oil,Bio fuel,Electric vehicle charging,Composting sawdust,Recycling center,zip_code,State
1869,NaN,Savannah International Trade and Convention Ce...,Informal Campsite,Quiet place right next to the parking lot of t...,32.0859,-81.0883,3.5926,2020-03-18 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,Yes,No,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,31401,GA
1870,NaN,Big branch refuge,Informal Campsite,Gravy parking lot before a national wildlife r...,30.2721,-89.8706,-7.6922,2018-04-13 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,70460,LA
1871,NaN,Monongahela NF - Gandy Creek,Wild Camping,Dispersed camping on Gandy Creek. Alongside r...,38.7809,-79.5618,918.8701,2019-07-17 00:00:00 UTC,Yes,No,No,No,nan,No,No,Natural Source,No,No,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,26296,WV
1872,NaN,Monongahela NF,Wild Camping,Grant’s Branch. Absolutely beautiful and peac...,38.7278,-79.6143,"1,034.2505",2018-04-13 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,No,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,26296,WV
1873,NaN,Durbin - Greenbrier River,Wild Camping,Quiet spot off the road amongst the rhododendr...,38.5273,-79.8409,825.5439,2018-08-02 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,No,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,26264,WV


In [60]:
USdata_1001 = USdata_need_loc.iloc[1000:,:]

In [61]:
zips, states = get_state_zip(USdata_1001)

Pulling request 2411, total API requests so far = 2739
waiting for: 3 seconds
Pulling request 2412, total API requests so far = 2740
waiting for: 1 seconds
Pulling request 2413, total API requests so far = 2741
waiting for: 3 seconds
Pulling request 2414, total API requests so far = 2742
waiting for: 2 seconds
Pulling request 2415, total API requests so far = 2743
waiting for: 2 seconds
Pulling request 2416, total API requests so far = 2744
waiting for: 3 seconds
Pulling request 2417, total API requests so far = 2745
waiting for: 2 seconds
Pulling request 2418, total API requests so far = 2746
waiting for: 1 seconds
Pulling request 2419, total API requests so far = 2747
waiting for: 1 seconds
Pulling request 2420, total API requests so far = 2748
waiting for: 1 seconds
Pulling request 2421, total API requests so far = 2749
waiting for: 1 seconds
Pulling request 2422, total API requests so far = 2750
waiting for: 2 seconds
Pulling request 2423, total API requests so far = 2751
waiting f

Pulling request 2517, total API requests so far = 2845
waiting for: 3 seconds
Pulling request 2518, total API requests so far = 2846
waiting for: 2 seconds
Pulling request 2519, total API requests so far = 2847
waiting for: 1 seconds
Pulling request 2520, total API requests so far = 2848
waiting for: 1 seconds
Pulling request 2521, total API requests so far = 2849
waiting for: 1 seconds
Pulling request 2522, total API requests so far = 2850
waiting for: 1 seconds
Pulling request 2523, total API requests so far = 2851
waiting for: 3 seconds
Pulling request 2524, total API requests so far = 2852
waiting for: 2 seconds
Pulling request 2525, total API requests so far = 2853
waiting for: 1 seconds
Pulling request 2526, total API requests so far = 2854
waiting for: 1 seconds
Pulling request 2527, total API requests so far = 2855
waiting for: 2 seconds
Pulling request 2528, total API requests so far = 2856
waiting for: 3 seconds
Pulling request 2529, total API requests so far = 2857
waiting f

waiting for: 1 seconds
Pulling request 2623, total API requests so far = 2951
waiting for: 3 seconds
Pulling request 2624, total API requests so far = 2952
waiting for: 1 seconds
Pulling request 2625, total API requests so far = 2953
waiting for: 3 seconds
Pulling request 2626, total API requests so far = 2954
waiting for: 3 seconds
Pulling request 2627, total API requests so far = 2955
waiting for: 3 seconds
Pulling request 2628, total API requests so far = 2956
waiting for: 3 seconds
Pulling request 2629, total API requests so far = 2957
waiting for: 2 seconds
Pulling request 2630, total API requests so far = 2958
waiting for: 3 seconds
Pulling request 2631, total API requests so far = 2959
waiting for: 2 seconds
Pulling request 2632, total API requests so far = 2960
waiting for: 3 seconds
Pulling request 2633, total API requests so far = 2961
waiting for: 3 seconds
Pulling request 2634, total API requests so far = 2962
waiting for: 1 seconds
Pulling request 2635, total API requests 

Pulling request 2728, total API requests so far = 3056
waiting for: 1 seconds
Pulling request 2729, total API requests so far = 3057
waiting for: 3 seconds
Pulling request 2730, total API requests so far = 3058
waiting for: 1 seconds
Pulling request 2731, total API requests so far = 3059
waiting for: 3 seconds
Pulling request 2732, total API requests so far = 3060
waiting for: 1 seconds
Pulling request 2733, total API requests so far = 3061
waiting for: 3 seconds
Pulling request 2734, total API requests so far = 3062
waiting for: 1 seconds
Pulling request 2735, total API requests so far = 3063
waiting for: 1 seconds
Pulling request 2736, total API requests so far = 3064
waiting for: 2 seconds
Pulling request 2737, total API requests so far = 3065
waiting for: 2 seconds
Pulling request 2738, total API requests so far = 3066
waiting for: 3 seconds
Pulling request 2739, total API requests so far = 3067
waiting for: 1 seconds
Pulling request 2740, total API requests so far = 3068
waiting f

Pulling request 2834, total API requests so far = 3162
waiting for: 3 seconds
Pulling request 2835, total API requests so far = 3163
waiting for: 3 seconds
Pulling request 2836, total API requests so far = 3164
waiting for: 1 seconds
Pulling request 2837, total API requests so far = 3165
waiting for: 2 seconds
Pulling request 2838, total API requests so far = 3166
waiting for: 2 seconds
Pulling request 2839, total API requests so far = 3167
waiting for: 3 seconds
Pulling request 2840, total API requests so far = 3168
waiting for: 1 seconds
Pulling request 2841, total API requests so far = 3169
waiting for: 2 seconds
Pulling request 2842, total API requests so far = 3170
waiting for: 2 seconds
Pulling request 2843, total API requests so far = 3171
waiting for: 1 seconds
Pulling request 2844, total API requests so far = 3172
waiting for: 1 seconds
Pulling request 2845, total API requests so far = 3173
waiting for: 2 seconds
Pulling request 2846, total API requests so far = 3174
waiting f

...

Pulling request 8339, total API requests so far = 8665
waiting for: 1 seconds
Pulling request 8340, total API requests so far = 8666
waiting for: 1 seconds
Pulling request 8341, total API requests so far = 8667
waiting for: 2 seconds
Pulling request 8342, total API requests so far = 8668
waiting for: 3 seconds
Pulling request 8343, total API requests so far = 8669
waiting for: 3 seconds
Pulling request 8344, total API requests so far = 8670
waiting for: 1 seconds
Pulling request 8345, total API requests so far = 8671
waiting for: 1 seconds
Pulling request 8346, total API requests so far = 8672
waiting for: 1 seconds
Pulling request 8347, total API requests so far = 8673
waiting for: 3 seconds
Pulling request 8348, total API requests so far = 8674
waiting for: 3 seconds
Pulling request 8349, total API requests so far = 8675
waiting for: 2 seconds
Pulling request 8350, total API requests so far = 8676
waiting for: 3 seconds
Pulling request 8351, total API requests so far = 8677
waiting f

Pulling request 8445, total API requests so far = 8771
waiting for: 1 seconds
Pulling request 8446, total API requests so far = 8772
waiting for: 1 seconds
Pulling request 8447, total API requests so far = 8773
waiting for: 2 seconds
Pulling request 8448, total API requests so far = 8774
waiting for: 2 seconds
Pulling request 8449, total API requests so far = 8775
waiting for: 1 seconds
Pulling request 8450, total API requests so far = 8776
waiting for: 2 seconds
Pulling request 8451, total API requests so far = 8777
waiting for: 1 seconds
Pulling request 8452, total API requests so far = 8778
waiting for: 3 seconds
Pulling request 8453, total API requests so far = 8779
waiting for: 1 seconds
Pulling request 8454, total API requests so far = 8780
waiting for: 3 seconds
Pulling request 8455, total API requests so far = 8781
waiting for: 3 seconds
Pulling request 8456, total API requests so far = 8782
waiting for: 2 seconds
Pulling request 8457, total API requests so far = 8783
waiting f

waiting for: 1 seconds
Pulling request 8551, total API requests so far = 8877
waiting for: 2 seconds
Pulling request 8552, total API requests so far = 8878
waiting for: 2 seconds
Pulling request 8553, total API requests so far = 8879
waiting for: 2 seconds
Pulling request 8554, total API requests so far = 8880
waiting for: 2 seconds
Pulling request 8555, total API requests so far = 8881
waiting for: 3 seconds
Pulling request 8556, total API requests so far = 8882
waiting for: 3 seconds
Pulling request 8557, total API requests so far = 8883
waiting for: 2 seconds
Pulling request 8558, total API requests so far = 8884
waiting for: 2 seconds
Pulling request 8559, total API requests so far = 8885
waiting for: 1 seconds
Pulling request 8560, total API requests so far = 8886
waiting for: 2 seconds
Pulling request 8561, total API requests so far = 8887
waiting for: 1 seconds
Pulling request 8562, total API requests so far = 8888
waiting for: 3 seconds
Pulling request 8563, total API requests 

Pulling request 8656, total API requests so far = 8982
waiting for: 3 seconds
Pulling request 8657, total API requests so far = 8983
waiting for: 3 seconds
Pulling request 8658, total API requests so far = 8984
waiting for: 3 seconds
Pulling request 8659, total API requests so far = 8985
waiting for: 2 seconds
Pulling request 8660, total API requests so far = 8986
waiting for: 3 seconds
Pulling request 8661, total API requests so far = 8987
waiting for: 2 seconds
Pulling request 8662, total API requests so far = 8988
waiting for: 1 seconds
Pulling request 8663, total API requests so far = 8989
waiting for: 2 seconds
Pulling request 8664, total API requests so far = 8990
waiting for: 3 seconds
Pulling request 8665, total API requests so far = 8991
waiting for: 3 seconds
Pulling request 8666, total API requests so far = 8992
waiting for: 3 seconds
Pulling request 8667, total API requests so far = 8993
waiting for: 3 seconds
Pulling request 8668, total API requests so far = 8994
waiting f

Pulling request 8762, total API requests so far = 9088
waiting for: 3 seconds
Pulling request 8763, total API requests so far = 9089
waiting for: 2 seconds
Pulling request 8764, total API requests so far = 9090
waiting for: 1 seconds
Pulling request 8765, total API requests so far = 9091
waiting for: 2 seconds
Pulling request 8766, total API requests so far = 9092
waiting for: 2 seconds
Pulling request 8767, total API requests so far = 9093
waiting for: 3 seconds
Pulling request 8768, total API requests so far = 9094
waiting for: 3 seconds
Pulling request 8769, total API requests so far = 9095
waiting for: 3 seconds
Pulling request 8770, total API requests so far = 9096
waiting for: 1 seconds
Pulling request 8771, total API requests so far = 9097
waiting for: 3 seconds
Pulling request 8772, total API requests so far = 9098
waiting for: 2 seconds
Pulling request 8773, total API requests so far = 9099
waiting for: 3 seconds
Pulling request 8774, total API requests so far = 9100
waiting f

In [62]:
# save
zips_pd_1001plus = pd.Series(zips, name='zip_codes') 
states_pd_1001plus = pd.Series(states, name='states') 
zips_pd_1001plus.to_csv('data/zip_codes_1001plus.csv')
states_pd_1001plus.to_csv('data/states_1001plus.csv')

In [63]:
len(zips_pd_1001plus)

6377

In [67]:
USdata_1001_zip_states = USdata_1001.copy()

In [68]:
USdata_1001_zip_states['zip_code'] = zips
USdata_1001_zip_states['State'] = states

In [69]:
USdata_1001_zip_states.head()


,Location,Name,Category,Description,Latitude,Longitude,Altitude,Date verified,Open,Electricity,Wifi,Kitchen,Parking,Restaurant,Showers,Water,Toilets,Big rig friendly,Tent friendly,Pet friendly,Sanitation dump station,Outdoor gear,Groceries,Artesian goods,Bakery,Rarity in this area,Repairs vehicles,Repairs motorcycles,Repairs bicycles,Sells parts,Recycles batteries,Recycles oil,Bio fuel,Electric vehicle charging,Composting sawdust,Recycling center,zip_code,State
2411,NaN,SHOWER - Smith Rock State Park,Showers,"Hot showers at the campground, $2, no time lim...",44.3611,-121.1408,863.5163,2020-06-25 00:00:00 UTC,Yes,NaN,NaN,NaN,nan,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,97760,OR
2412,NaN,Salida Hot Springs Aquatic Center,Showers,"Place to take a shower for $5, no time limit, ...",38.5245,-106.0094,"2,180.8336",2019-11-23 16:56:09 UTC,Yes,NaN,NaN,NaN,nan,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,81201,CO
2413,NaN,Well water at campground,Water,"Assuming potable- Locals filling up at site, o...",61.7784,-149.1978,555.2069,2019-08-01 00:00:00 UTC,Yes,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,99645,AK
2414,NaN,Community medical center & Park,Informal Campsite,"We sleep here 2 night. Quiet place, may be som...",60.5412,-145.7520,22.8404,2018-07-19 00:00:00 UTC,Yes,No,No,No,nan,No,No,Yes,Running Water,Yes,No,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,99574,AK
2415,NaN,Elk River bank large camp spot,Wild Camping,"Going up the Elk River road, this is a nice sp...",42.7118,-124.3744,62.1792,2020-06-17 00:00:00 UTC,Yes,No,No,No,nan,No,No,Natural Source,No,No,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,97465,OR


In [70]:
# concat together
USdata_1001_zip_states.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6377 entries, 2411 to 8789
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Location                   0 non-null      object 
 1   Name                       6377 non-null   object 
 2   Category                   6377 non-null   object 
 3   Description                6377 non-null   object 
 4   Latitude                   6377 non-null   float64
 5   Longitude                  6377 non-null   float64
 6   Altitude                   6199 non-null   float64
 7   Date verified              6377 non-null   object 
 8   Open                       6377 non-null   object 
 9   Electricity                5678 non-null   object 
 10  Wifi                       5678 non-null   object 
 11  Kitchen                    5678 non-null   object 
 12  Parking                    0 non-null      float64
 13  Restaurant                 5678 non-null   ob

In [71]:
USdata_501_1000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 1869 to 2410
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Location                   0 non-null      object 
 1   Name                       500 non-null    object 
 2   Category                   500 non-null    object 
 3   Description                500 non-null    object 
 4   Latitude                   500 non-null    float64
 5   Longitude                  500 non-null    float64
 6   Altitude                   495 non-null    float64
 7   Date verified              500 non-null    object 
 8   Open                       500 non-null    object 
 9   Electricity                479 non-null    object 
 10  Wifi                       479 non-null    object 
 11  Kitchen                    479 non-null    object 
 12  Parking                    0 non-null      float64
 13  Restaurant                 479 non-null    obj

In [72]:
USdata_zip_states.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 39 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   index                      500 non-null    int64  
 1   Location                   0 non-null      object 
 2   Name                       500 non-null    object 
 3   Category                   500 non-null    object 
 4   Description                500 non-null    object 
 5   Latitude                   500 non-null    float64
 6   Longitude                  500 non-null    float64
 7   Altitude                   500 non-null    float64
 8   Date verified              500 non-null    object 
 9   Open                       500 non-null    object 
 10  Electricity                469 non-null    object 
 11  Wifi                       469 non-null    object 
 12  Kitchen                    469 non-null    object 
 13  Parking                    0 non-null      float64

In [73]:
USdata_zip_states.head()

,index,Location,Name,Category,Description,Latitude,Longitude,Altitude,Date verified,Open,Electricity,Wifi,Kitchen,Parking,Restaurant,Showers,Water,Toilets,Big rig friendly,Tent friendly,Pet friendly,Sanitation dump station,Outdoor gear,Groceries,Artesian goods,Bakery,Rarity in this area,Repairs vehicles,Repairs motorcycles,Repairs bicycles,Sells parts,Recycles batteries,Recycles oil,Bio fuel,Electric vehicle charging,Composting sawdust,Recycling center,zip_code,State
0,1221,NaN,Springer Lake,Wild Camping,Dispersed camping on shore of small lake. Ceme...,36.4088,-104.6526,"1,788.0000",2020-04-01 00:00:00 UTC,Yes,No,No,No,nan,No,No,Natural Source,Pit Toilets,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,87747,NM
1,1222,NaN,"large flat spot - on route 9 , before the 89 j...",Wild Camping,Be careful low branches. Not suitable for rvs...,37.2615,-112.7791,"1,811.0000",2019-05-29 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,84755,UT
2,1223,NaN,Zigzag falls,Wild Camping,Lots of small camping spots along Kiwanis Camp...,45.3141,-121.8082,944.3531,2019-06-02 00:00:00 UTC,Yes,No,No,No,nan,No,No,Natural Source,No,No,Yes,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,97041,OR
3,1224,NaN,Topsy Grade Rd,Wild Camping,Open woodlands along the reservoir with plenty...,42.1318,-122.0332,"1,194.4113",2019-09-27 00:00:00 UTC,Yes,No,No,No,nan,No,No,Non-Potable,No,No,Yes,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,97627,OR
4,1225,NaN,Lava tubes,Wild Camping,Pit toilet picnic table site,34.9446,-108.1069,"2,324.8057",2017-09-24 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,Pit Toilets,Unknown,Unknown,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,87020,NM


In [75]:
USdata_zip_states.set_index('index', inplace=True)

In [76]:
USdata_zip_states.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 1221 to 1868
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Location                   0 non-null      object 
 1   Name                       500 non-null    object 
 2   Category                   500 non-null    object 
 3   Description                500 non-null    object 
 4   Latitude                   500 non-null    float64
 5   Longitude                  500 non-null    float64
 6   Altitude                   500 non-null    float64
 7   Date verified              500 non-null    object 
 8   Open                       500 non-null    object 
 9   Electricity                469 non-null    object 
 10  Wifi                       469 non-null    object 
 11  Kitchen                    469 non-null    object 
 12  Parking                    0 non-null      float64
 13  Restaurant                 469 non-null    obj

In [77]:
USdata_zip_states.head()

,Location,Name,Category,Description,Latitude,Longitude,Altitude,Date verified,Open,Electricity,Wifi,Kitchen,Parking,Restaurant,Showers,Water,Toilets,Big rig friendly,Tent friendly,Pet friendly,Sanitation dump station,Outdoor gear,Groceries,Artesian goods,Bakery,Rarity in this area,Repairs vehicles,Repairs motorcycles,Repairs bicycles,Sells parts,Recycles batteries,Recycles oil,Bio fuel,Electric vehicle charging,Composting sawdust,Recycling center,zip_code,State
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1221,NaN,Springer Lake,Wild Camping,Dispersed camping on shore of small lake. Ceme...,36.4088,-104.6526,"1,788.0000",2020-04-01 00:00:00 UTC,Yes,No,No,No,nan,No,No,Natural Source,Pit Toilets,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,87747,NM
1222,NaN,"large flat spot - on route 9 , before the 89 j...",Wild Camping,Be careful low branches. Not suitable for rvs...,37.2615,-112.7791,"1,811.0000",2019-05-29 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,84755,UT
1223,NaN,Zigzag falls,Wild Camping,Lots of small camping spots along Kiwanis Camp...,45.3141,-121.8082,944.3531,2019-06-02 00:00:00 UTC,Yes,No,No,No,nan,No,No,Natural Source,No,No,Yes,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,97041,OR
1224,NaN,Topsy Grade Rd,Wild Camping,Open woodlands along the reservoir with plenty...,42.1318,-122.0332,"1,194.4113",2019-09-27 00:00:00 UTC,Yes,No,No,No,nan,No,No,Non-Potable,No,No,Yes,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,97627,OR
1225,NaN,Lava tubes,Wild Camping,Pit toilet picnic table site,34.9446,-108.1069,"2,324.8057",2017-09-24 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,Pit Toilets,Unknown,Unknown,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,87020,NM


In [78]:
dfs = [USdata_zip_states, USdata_501_1000, USdata_1001_zip_states]

In [79]:
USdata_zip_states_combined = pd.concat(dfs)

In [80]:
USdata_zip_states_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7377 entries, 1221 to 8789
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Location                   0 non-null      object 
 1   Name                       7377 non-null   object 
 2   Category                   7377 non-null   object 
 3   Description                7377 non-null   object 
 4   Latitude                   7377 non-null   float64
 5   Longitude                  7377 non-null   float64
 6   Altitude                   7194 non-null   float64
 7   Date verified              7377 non-null   object 
 8   Open                       7377 non-null   object 
 9   Electricity                6626 non-null   object 
 10  Wifi                       6626 non-null   object 
 11  Kitchen                    6626 non-null   object 
 12  Parking                    0 non-null      float64
 13  Restaurant                 6626 non-null   ob

In [81]:
USdata_has_loc = USdata[~pd.isna(USdata['Location'])].copy()

In [86]:
USdata_has_loc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1413 entries, 0 to 5636
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Location                   1413 non-null   object 
 1   Name                       1413 non-null   object 
 2   Category                   1413 non-null   object 
 3   Description                1413 non-null   object 
 4   Latitude                   1413 non-null   float64
 5   Longitude                  1413 non-null   float64
 6   Altitude                   1373 non-null   float64
 7   Date verified              1413 non-null   object 
 8   Open                       1413 non-null   object 
 9   Electricity                1361 non-null   object 
 10  Wifi                       1361 non-null   object 
 11  Kitchen                    1361 non-null   object 
 12  Parking                    0 non-null      float64
 13  Restaurant                 1361 non-null   objec

In [83]:
USdata_has_loc.head()

,Location,Name,Category,Description,Latitude,Longitude,Altitude,Date verified,Open,Electricity,Wifi,Kitchen,Parking,Restaurant,Showers,Water,Toilets,Big rig friendly,Tent friendly,Pet friendly,Sanitation dump station,Outdoor gear,Groceries,Artesian goods,Bakery,Rarity in this area,Repairs vehicles,Repairs motorcycles,Repairs bicycles,Sells parts,Recycles batteries,Recycles oil,Bio fuel,Electric vehicle charging,Composting sawdust,Recycling center
0,"Borrego Salton Seaway, Borrego Springs, CA 920...",Anza Borrego - NICE,Wild Camping,Free Camping on Anza Borrego Park. Nice and wa...,33.2806,-116.1469,0.0000,2019-01-28 00:00:00 UTC,Yes,Unknown,Unknown,Unknown,nan,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
1,"Unnamed Road, Austin, NV 89310, USA",Spencer Hotsprings,Wild Camping,Hammer site. Lots of places for vehicle or ten...,39.3306,-116.8546,"1,707.0000",2020-07-03 00:00:00 UTC,Yes,No,No,No,nan,No,No,Natural Source,No,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
2,"Burr Trail Rd, Boulder, UT 84716, USA",Burr Trail - NICE,Wild Camping,Very quiet sleeping places on the Burr Trail t...,37.8702,-111.3375,"1,805.2075",2017-03-25 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,Unknown,Unknown,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
3,"Coral Pink Sand Dunes Rd, Kanab, UT 84741, USA",Coral Pink Sand Dunes - OK,Wild Camping,Free camping without nothing in the BLM area. ...,37.0682,-112.7028,nan,2012-01-01 00:00:00 UTC,Yes,Unknown,Unknown,Unknown,nan,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
4,"Lincoln Highway, Ruth, NV 89319, USA",Ely - OK,Wild Camping,"Cookies at the 50 to Ely. Heard BLM. Fire pit,...",39.3104,-114.9900,"2,079.7598",2019-06-11 18:32:16 UTC,Yes,No,No,No,nan,No,No,No,Unknown,No,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
USdata_has_loc[USdata_has_loc['Location'].str.contains(' CO', case=True, regex=False)]

,Location,Name,Category,Description,Latitude,Longitude,Altitude,Date verified,Open,Electricity,Wifi,Kitchen,Parking,Restaurant,Showers,Water,Toilets,Big rig friendly,Tent friendly,Pet friendly,Sanitation dump station,Outdoor gear,Groceries,Artesian goods,Bakery,Rarity in this area,Repairs vehicles,Repairs motorcycles,Repairs bicycles,Sells parts,Recycles batteries,Recycles oil,Bio fuel,Electric vehicle charging,Composting sawdust,Recycling center
13,"San Juan National Forest, Usfs 622, Bayfield, ...",National Forest Outside Durango,Wild Camping,Free. Great quiet camping in canyon next to r...,37.3555,-107.3236,"2,221.0000",2014-06-16 10:07:52 UTC,Yes,No,No,No,nan,No,No,Natural Source,Pit Toilets,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
14,"United States Forest Grand Mesa, 8001-8227 Co ...",National Forest outside Ouray,Wild Camping,Beautiful mountain scenery along this 40 mile ...,38.2037,-107.5117,"2,765.0000",2014-06-19 08:17:02 UTC,Yes,No,No,No,nan,No,No,Natural Source,No,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
16,"330 East Road, Silt, CO 81652, USA",National Forest outside Glenwood Springs,Wild Camping,Free. National Forest camping off a dirt road...,39.3312,-107.6682,"2,437.0000",2019-09-26 16:45:26 UTC,Yes,No,No,No,nan,No,No,No,No,No,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
17,"Co Rd 67, Maybell, CO 81640, USA",Wild Horse Area,Wild Camping,Free camping in the middle of the Sand Wash Wi...,40.6797,-108.3990,"1,862.0000",2018-12-03 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
101,"33993 US-160, Mancos, CO 81328, USA",Mesa verde Nationalpark (outside In BLM),Wild Camping,Close to the entrence of mesa verde nationalpa...,37.3595,-108.4228,"2,054.2000",2020-07-20 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,No,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1303,"Unnamed Road, Idalia, CO 80735, USA","North Cove, Bonny Lake State Park",Informal Campsite,Abandoned campground near empty reservoir. Fre...,39.6383,-102.1869,"1,110.0000",2019-11-07 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,Pit Toilets,Yes,Yes,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
1479,"20205 CO-115, Florence, CO 81226, USA",Pathfinder Regional Park in Florence,Informal Campsite,A large recreational park close to the Arkansa...,38.4068,-105.1420,"1,611.0604",2019-05-16 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,Running Water,Yes,No,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
1533,"Lake Como Rd, Mosca, CO 81146, USA",Lake Como Rd boondocking,Wild Camping,Boondocking off this road which leads up to La...,37.5355,-105.5804,"2,391.0000",2019-11-22 11:14:21 UTC,Yes,No,No,No,nan,No,No,No,No,Unknown,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
1534,"Usfs 667, Pagosa Springs, CO 81147, USA",East Fork Rd boondocking,Wild Camping,Gorgeous boondocking site! Continue on East F...,37.3896,-106.8434,"2,432.0000",2019-08-09 10:37:09 UTC,Yes,No,No,No,nan,No,No,Natural Source,No,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
USdata_has_loc[USdata_has_loc['Location'].str.contains('\s[A-Z]{2}\s', case=True, regex=True)]

,Location,Name,Category,Description,Latitude,Longitude,Altitude,Date verified,Open,Electricity,Wifi,Kitchen,Parking,Restaurant,Showers,Water,Toilets,Big rig friendly,Tent friendly,Pet friendly,Sanitation dump station,Outdoor gear,Groceries,Artesian goods,Bakery,Rarity in this area,Repairs vehicles,Repairs motorcycles,Repairs bicycles,Sells parts,Recycles batteries,Recycles oil,Bio fuel,Electric vehicle charging,Composting sawdust,Recycling center
0,"Borrego Salton Seaway, Borrego Springs, CA 920...",Anza Borrego - NICE,Wild Camping,Free Camping on Anza Borrego Park. Nice and wa...,33.2806,-116.1469,0.0000,2019-01-28 00:00:00 UTC,Yes,Unknown,Unknown,Unknown,nan,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
1,"Unnamed Road, Austin, NV 89310, USA",Spencer Hotsprings,Wild Camping,Hammer site. Lots of places for vehicle or ten...,39.3306,-116.8546,"1,707.0000",2020-07-03 00:00:00 UTC,Yes,No,No,No,nan,No,No,Natural Source,No,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
2,"Burr Trail Rd, Boulder, UT 84716, USA",Burr Trail - NICE,Wild Camping,Very quiet sleeping places on the Burr Trail t...,37.8702,-111.3375,"1,805.2075",2017-03-25 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,Unknown,Unknown,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
3,"Coral Pink Sand Dunes Rd, Kanab, UT 84741, USA",Coral Pink Sand Dunes - OK,Wild Camping,Free camping without nothing in the BLM area. ...,37.0682,-112.7028,nan,2012-01-01 00:00:00 UTC,Yes,Unknown,Unknown,Unknown,nan,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
4,"Lincoln Highway, Ruth, NV 89319, USA",Ely - OK,Wild Camping,"Cookies at the 50 to Ely. Heard BLM. Fire pit,...",39.3104,-114.9900,"2,079.7598",2019-06-11 18:32:16 UTC,Yes,No,No,No,nan,No,No,No,Unknown,No,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2067,"I-15, Bunkerville, NV 89007, USA",Truck parking,Informal Campsite,"A truck parking, without anything. Only a park...",36.7778,-114.2661,654.6721,2018-05-20 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,Pit Toilets,Yes,No,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
2072,"Mexican Hat Rock Rd, Mexican Hat, UT 84531, USA",Between San Juan river and Mexican Hat,Wild Camping,"A lot near the road along San Juan river, real...",37.1757,-109.8456,0.0000,2020-05-20 00:00:00 UTC,Yes,Unknown,Unknown,Unknown,nan,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
2091,"818-898 E Locust St, San Antonio, TX 78212, USA",Neighborhood Parking,Wild Camping,Super quiet and low profile place to park for ...,29.4439,-98.4824,0.0000,2020-03-12 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,Unknown,No,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
2140,"Leeds, ME 04263, USA",Pull out to river,Wild Camping,"a small piece of asfalt down to the river, lea...",44.3452,-70.1493,nan,2018-05-29 00:00:00 UTC,Yes,No,No,No,nan,No,No,Natural Source,No,No,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
USdata_has_loc[~USdata_has_loc['Location'].str.contains('\s[A-Z]{2}\s', case=True, regex=True)]

,Location,Name,Category,Description,Latitude,Longitude,Altitude,Date verified,Open,Electricity,Wifi,Kitchen,Parking,Restaurant,Showers,Water,Toilets,Big rig friendly,Tent friendly,Pet friendly,Sanitation dump station,Outdoor gear,Groceries,Artesian goods,Bakery,Rarity in this area,Repairs vehicles,Repairs motorcycles,Repairs bicycles,Sells parts,Recycles batteries,Recycles oil,Bio fuel,Electric vehicle charging,Composting sawdust,Recycling center
10,"Smoky Mountain Rd, Utah, USA",Smokey Mountain Rd - NICE,Wild Camping,Very nice place on top of the cliff with great...,37.1879,-111.4515,"1,719.2568",2020-04-22 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,No,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
15,"Gunnison National Forest, 11088-11860 Co Road ...",National Forest outside Crested Butte,Wild Camping,"Free. Beautiful mountain scenery, 15 minutes o...",38.8578,-107.1550,"2,704.0000",2019-08-23 09:11:05 UTC,Yes,No,No,No,nan,No,No,Natural Source,No,No,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
18,"Forest Rd 539, Utah, USA",Flaming Gorge national Forest,Wild Camping,Free wild camping in Flaming Gorge. Great mou...,40.8486,-109.7061,"2,481.0000",2018-08-24 08:45:35 UTC,Yes,No,No,No,nan,No,No,No,No,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
21,"US-95, Nevada, USA","Rest Stop After Tonopah, NV",Informal Campsite,"Rest area with basic amenities: restrooms, pic...",38.1406,-117.4538,"1,418.0000",2019-09-18 00:00:00 UTC,Yes,No,No,No,nan,No,No,Yes,Yes,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
24,"United States, North Carolina, Montgomery",Top of Daniel at Uwharrie National Forest,Wild Camping,Free site since its National Forest land. Take...,35.4181,-80.0331,267.4548,2014-10-07 09:12:20 UTC,Yes,No,No,No,nan,No,No,No,No,No,Yes,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,"Kelso-Dunes Rd, California, USA",Kelso Dunes,Informal Campsite,HUGE camping area with tons of space and fire ...,34.8882,-115.7165,768.8718,2020-02-14 15:32:48 UTC,Yes,No,No,No,nan,No,No,No,No,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
1700,"Cam Playa, Aguada, 00602, Puerto Rico",Pico de Piedra Beach,Informal Campsite,"Nice and Beautiful west coast beach , Great Re...",18.3842,-67.2122,nan,2017-12-22 00:00:00 UTC,Yes,Yes,Yes - Average,Yes,nan,Yes,Warm,Potable,Running Water,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
1891,"Unnamed Road, Texas, USA","Williams Road to Gypsum Dunes, Dell City",Wild Camping,Wild Camping in the desert along Wilson Road l...,31.9211,-105.0794,"1,066.0000",2020-03-17 00:00:00 UTC,Yes,Unknown,Unknown,Unknown,nan,Unknown,Unknown,Unknown,Unknown,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN
5582,"Соединённые Штаты Америки, штат Аризона, Кокон...",Forest,Wild Camping,Many bivouac sites on this road for both vr an...,35.1780,-111.4994,"2,009.2375",2019-08-10 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
USdata_has_loc = USdata_has_loc[USdata_has_loc['Location'].str.contains('\s[A-Z]{2}\s', case=True, regex=True)]

In [89]:
USdata_has_loc['zip_code'] = ''

In [97]:
USdata_has_loc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1295 entries, 0 to 2146
Data columns (total 37 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Location                   1295 non-null   object 
 1   Name                       1295 non-null   object 
 2   Category                   1295 non-null   object 
 3   Description                1295 non-null   object 
 4   Latitude                   1295 non-null   float64
 5   Longitude                  1295 non-null   float64
 6   Altitude                   1265 non-null   float64
 7   Date verified              1295 non-null   object 
 8   Open                       1295 non-null   object 
 9   Electricity                1243 non-null   object 
 10  Wifi                       1243 non-null   object 
 11  Kitchen                    1243 non-null   object 
 12  Parking                    0 non-null      float64
 13  Restaurant                 1243 non-null   objec

In [96]:
USdata_has_loc['Location'][0]

'Borrego Salton Seaway, Borrego Springs, CA 92004, USA'

In [104]:
USdata_has_loc['State'] = USdata_has_loc['Location'].str.extract(r'(\s[A-Z]{2}\s)')

In [105]:
USdata_has_loc.head()

,Location,Name,Category,Description,Latitude,Longitude,Altitude,Date verified,Open,Electricity,Wifi,Kitchen,Parking,Restaurant,Showers,Water,Toilets,Big rig friendly,Tent friendly,Pet friendly,Sanitation dump station,Outdoor gear,Groceries,Artesian goods,Bakery,Rarity in this area,Repairs vehicles,Repairs motorcycles,Repairs bicycles,Sells parts,Recycles batteries,Recycles oil,Bio fuel,Electric vehicle charging,Composting sawdust,Recycling center,zip_code,State
0,"Borrego Salton Seaway, Borrego Springs, CA 920...",Anza Borrego - NICE,Wild Camping,Free Camping on Anza Borrego Park. Nice and wa...,33.2806,-116.1469,0.0000,2019-01-28 00:00:00 UTC,Yes,Unknown,Unknown,Unknown,nan,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,,CA
1,"Unnamed Road, Austin, NV 89310, USA",Spencer Hotsprings,Wild Camping,Hammer site. Lots of places for vehicle or ten...,39.3306,-116.8546,"1,707.0000",2020-07-03 00:00:00 UTC,Yes,No,No,No,nan,No,No,Natural Source,No,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,,NV
2,"Burr Trail Rd, Boulder, UT 84716, USA",Burr Trail - NICE,Wild Camping,Very quiet sleeping places on the Burr Trail t...,37.8702,-111.3375,"1,805.2075",2017-03-25 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,Unknown,Unknown,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,,UT
3,"Coral Pink Sand Dunes Rd, Kanab, UT 84741, USA",Coral Pink Sand Dunes - OK,Wild Camping,Free camping without nothing in the BLM area. ...,37.0682,-112.7028,nan,2012-01-01 00:00:00 UTC,Yes,Unknown,Unknown,Unknown,nan,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,,UT
4,"Lincoln Highway, Ruth, NV 89319, USA",Ely - OK,Wild Camping,"Cookies at the 50 to Ely. Heard BLM. Fire pit,...",39.3104,-114.9900,"2,079.7598",2019-06-11 18:32:16 UTC,Yes,No,No,No,nan,No,No,No,Unknown,No,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,,NV


In [106]:
USdata_has_loc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1295 entries, 0 to 2146
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Location                   1295 non-null   object 
 1   Name                       1295 non-null   object 
 2   Category                   1295 non-null   object 
 3   Description                1295 non-null   object 
 4   Latitude                   1295 non-null   float64
 5   Longitude                  1295 non-null   float64
 6   Altitude                   1265 non-null   float64
 7   Date verified              1295 non-null   object 
 8   Open                       1295 non-null   object 
 9   Electricity                1243 non-null   object 
 10  Wifi                       1243 non-null   object 
 11  Kitchen                    1243 non-null   object 
 12  Parking                    0 non-null      float64
 13  Restaurant                 1243 non-null   objec

In [107]:
dfs.append(USdata_has_loc)

In [108]:
dfs

[      Location                                               Name  \
 index                                                               
 1221       NaN                                      Springer Lake   
 1222       NaN  large flat spot - on route 9 , before the 89 j...   
 1223       NaN                                       Zigzag falls   
 1224       NaN                                     Topsy Grade Rd   
 1225       NaN                                         Lava tubes   
 ...        ...                                                ...   
 1862       NaN      Veterans Memorial Park and area, Lake Charles   
 1863       NaN                                  Street near beach   
 1866       NaN                       Off of hole in the rock road   
 1867       NaN                      Free campsite in Coconico NF    
 1868       NaN  Chevron gas station with very large paved over...   
 
                 Category                                        Description  \
 index  

In [109]:
USdata_zip_states_combined = pd.concat(dfs)

In [110]:
USdata_zip_states_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8672 entries, 1221 to 2146
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Location                   1295 non-null   object 
 1   Name                       8672 non-null   object 
 2   Category                   8672 non-null   object 
 3   Description                8672 non-null   object 
 4   Latitude                   8672 non-null   float64
 5   Longitude                  8672 non-null   float64
 6   Altitude                   8459 non-null   float64
 7   Date verified              8672 non-null   object 
 8   Open                       8672 non-null   object 
 9   Electricity                7869 non-null   object 
 10  Wifi                       7869 non-null   object 
 11  Kitchen                    7869 non-null   object 
 12  Parking                    0 non-null      float64
 13  Restaurant                 7869 non-null   ob

In [111]:
USdata_zip_states_combined.head()

,Location,Name,Category,Description,Latitude,Longitude,Altitude,Date verified,Open,Electricity,Wifi,Kitchen,Parking,Restaurant,Showers,Water,Toilets,Big rig friendly,Tent friendly,Pet friendly,Sanitation dump station,Outdoor gear,Groceries,Artesian goods,Bakery,Rarity in this area,Repairs vehicles,Repairs motorcycles,Repairs bicycles,Sells parts,Recycles batteries,Recycles oil,Bio fuel,Electric vehicle charging,Composting sawdust,Recycling center,zip_code,State
1221,NaN,Springer Lake,Wild Camping,Dispersed camping on shore of small lake. Ceme...,36.4088,-104.6526,"1,788.0000",2020-04-01 00:00:00 UTC,Yes,No,No,No,nan,No,No,Natural Source,Pit Toilets,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,87747,NM
1222,NaN,"large flat spot - on route 9 , before the 89 j...",Wild Camping,Be careful low branches. Not suitable for rvs...,37.2615,-112.7791,"1,811.0000",2019-05-29 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,No,Yes,Yes,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,84755,UT
1223,NaN,Zigzag falls,Wild Camping,Lots of small camping spots along Kiwanis Camp...,45.3141,-121.8082,944.3531,2019-06-02 00:00:00 UTC,Yes,No,No,No,nan,No,No,Natural Source,No,No,Yes,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,97041,OR
1224,NaN,Topsy Grade Rd,Wild Camping,Open woodlands along the reservoir with plenty...,42.1318,-122.0332,"1,194.4113",2019-09-27 00:00:00 UTC,Yes,No,No,No,nan,No,No,Non-Potable,No,No,Yes,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,97627,OR
1225,NaN,Lava tubes,Wild Camping,Pit toilet picnic table site,34.9446,-108.1069,"2,324.8057",2017-09-24 00:00:00 UTC,Yes,No,No,No,nan,No,No,No,Pit Toilets,Unknown,Unknown,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,87020,NM


In [112]:
USdata_zip_states_combined.to_csv('data/USdata_zip_states_combined.csv')

In [113]:
USdata_zip_states_combined['State'].unique()

array(['NM', 'UT', 'OR', 'WA', 'CO', 'CA', 'IA', 'ME', 'NC', 'ID', 'AZ',
       'NV', 'MT', 'NH', 'WY', 'TN', 'SD', 'MI', 'MD', '', 'FL', 'TX',
       'OK', 'VT', 'MO', 'WI', 'MN', 'KS', 'ND', 'NE', 'GA', 'LA', 'SC',
       'AK', 'CT', 'MS', 'VA', 'PA', 'MA', 'NJ', 'KY', 'WV', 'NY', 'IL',
       'DC', 'AL', 'ON', 'OH', 'DE', 'QC', 'AR', 'NB', 'IN', 'HI', 'AB',
       'BC', 'RI', 'S.L.P.', 'Ver.', 'Nay.', 'YT', 'NS', 'B.C.', 'Pue.',
       ' CA ', ' NV ', ' UT ', ' AZ ', ' CO ', ' WV ', ' AL ', ' WA ',
       ' OR ', ' TX ', ' NM ', ' LA ', ' NC ', ' WY ', ' PA ', ' AK ',
       ' MT ', ' TN ', ' ID ', ' FL ', ' GA ', ' SC ', ' SE ', ' MS ',
       ' VA ', ' MD ', ' VT ', ' IL ', ' NY ', ' ME ', ' SD ', ' ND ',
       ' OK ', ' KS ', ' NE ', ' KY ', ' MN ', ' IA ', ' NJ ', ' DC ',
       ' MA ', ' MO ', ' US ', ' NW ', ' BC ', ' WI ', ' AR ', ' OH ',
       ' QC ', ' MI ', ' AB ', ' SW ', ' NS ', ' IN '], dtype=object)

In [115]:
USdata_zip_states_combined['State'] = USdata_zip_states_combined['State'].str.strip()

In [122]:
states = USdata_zip_states_combined['State'].unique()

In [123]:
len(states)

68

In [124]:
USdata_CO = USdata_zip_states_combined[USdata_zip_states_combined['State'] == 'CO']

In [125]:
USdata_CO.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 550 entries, 1227 to 1852
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Location                   109 non-null    object 
 1   Name                       550 non-null    object 
 2   Category                   550 non-null    object 
 3   Description                550 non-null    object 
 4   Latitude                   550 non-null    float64
 5   Longitude                  550 non-null    float64
 6   Altitude                   535 non-null    float64
 7   Date verified              550 non-null    object 
 8   Open                       550 non-null    object 
 9   Electricity                502 non-null    object 
 10  Wifi                       502 non-null    object 
 11  Kitchen                    502 non-null    object 
 12  Parking                    0 non-null      float64
 13  Restaurant                 502 non-null    obj

In [132]:
USdata_zip_states_combined.groupby('State').agg('count').sort_values(by='Description', ascending=False).head(10)

,Location,Name,Category,Description,Latitude,Longitude,Altitude,Date verified,Open,Electricity,Wifi,Kitchen,Parking,Restaurant,Showers,Water,Toilets,Big rig friendly,Tent friendly,Pet friendly,Sanitation dump station,Outdoor gear,Groceries,Artesian goods,Bakery,Rarity in this area,Repairs vehicles,Repairs motorcycles,Repairs bicycles,Sells parts,Recycles batteries,Recycles oil,Bio fuel,Electric vehicle charging,Composting sawdust,Recycling center,zip_code
State,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
CA,204,1223,1223,1223,1223,1223,1196,1223,1223,1081,1081,1081,0,1081,1126,1081,1081,1081,1081,1081,0,0,0,0,0,0,0,0,0,0,3,3,3,3,3,3,1223
AZ,120,682,682,682,682,682,672,682,682,602,602,602,0,602,622,602,602,602,602,602,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,682
CO,109,550,550,550,550,550,535,550,550,502,502,502,0,502,515,502,502,502,502,502,0,0,0,0,0,0,0,0,0,0,6,6,6,6,6,6,550
,0,546,546,546,546,546,533,546,546,531,531,531,0,531,535,531,531,531,531,531,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,546
OR,76,525,525,525,525,525,522,525,525,482,482,482,0,482,500,482,482,482,482,482,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,525
UT,101,490,490,490,490,490,483,490,490,424,424,424,0,424,443,424,424,424,424,424,0,0,0,0,0,0,0,0,0,0,7,7,7,7,7,7,490
WA,53,485,485,485,485,485,478,485,485,445,445,445,0,445,469,445,445,445,445,445,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,485
TX,48,331,331,331,331,331,319,331,331,301,301,301,0,301,307,301,301,301,301,301,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,331
AK,137,316,316,316,316,316,305,316,316,281,281,281,0,281,294,281,281,281,281,281,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,316


In [134]:
USdata_zip_states_combined.to_csv('data/USdata_zip_states_combined_cleaned.csv', index=False)